# Load multiple COGs into xarray dataset

In [ ]:
import satsearch
from satstac import Items
import geopandas as gpd

In [ ]:
gfa = gpd.read_file('aoi.geojson')
gfa

In [ ]:
bbox = gfa.bounds.values[0]
west, north, east, south = bbox

In [ ]:
properties =  ["landsat:tier=T1"] 

bbox = (west, south, east, north) #(min lon, min lat, max lon, max lat)

results = satsearch.Search.search(collection='landsat-8-l1', 
                        bbox=bbox, 
                        datetime='2019-07-15/2019-07-31',
                        sort=['<datetime'], #earliest scene first
                        property=properties)
print('%s items' % results.found())

In [ ]:
results.found()

In [ ]:
import holoviews as hv
import hvplot.xarray
import hvplot.pandas
import geoviews as gv

In [ ]:
items = results.items()
items.save('items-landsat8.json')
items = Items.load('items-landsat8.json')
#items.bbox()

In [ ]:
gfl = gpd.read_file('items-landsat8.json')
gfl = gfl.sort_values('datetime').reset_index(drop=True)
print('records:', len(gfl))
gfl.head()

In [ ]:
# Plot search AOI and frames on a map

# just keep id for hover tips
cols = gfl.loc[:,('id','geometry')]

footprints = cols.hvplot(geo=True, line_color='k', alpha=0.1, title='Landsat 8 T1')
aoi = gfa.hvplot(geo=True, line_color='b', fill_color=None)
tiles = gv.tile_sources.CartoEco.options(width=700, height=500) #.redim.range(Latitude=(45, 50), Longitude=(-126,-120)) 
labels = gv.tile_sources.StamenLabels.options(level='annotation')
tiles * footprints * aoi * labels

In [ ]:
# Load these into an xarray dataset
# URLS of these 4 images
da1 = xr.open_rasterio('top.tif')
da2 = xr.open_rasterio('bottom.tif')